In [66]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

In [67]:
max_features = 10000
(X_train,y_train),(X_test,y_test)= imdb.load_data(num_words=max_features)

In [68]:
#mapping of words index back to words(for understanding)
d = {value:key for key,value in imdb.get_word_index().items()}
sample = [d.get(i-3,'and') for i in X_train[0]]

In [69]:
from tensorflow.keras.utils import pad_sequences
sent_len=500
X_train=pad_sequences(X_train,padding='pre',maxlen=sent_len)
X_test=pad_sequences(X_test,padding='pre',maxlen=sent_len)

In [70]:
model = Sequential()
# Example with GloVe embedding (if you download the embeddings)
embedding_matrix = np.load('glove_embedding.npy')  # Example
model.add(Embedding(max_features, 128, weights=[embedding_matrix], trainable=False))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [71]:
model.compile('adam','binary_crossentropy',metrics=["accuracy"])

In [72]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [73]:
history = model.fit(
    X_train,y_train,epochs=50,batch_size=32,validation_split=0.2,callbacks=[es]
)

Epoch 1/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 65ms/step - accuracy: 0.6132 - loss: 19.0771 - val_accuracy: 0.6150 - val_loss: 0.6365
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 54s 86ms/step - accuracy: 0.7106 - loss: 0.5626 - val_accuracy: 0.8158 - val_loss: 0.4146
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 96ms/step - accuracy: 0.7909 - loss: 0.4711 - val_accuracy: 0.7514 - val_loss: 0.4969
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 59s 95ms/step - accuracy: 0.8668 - loss: 0.3396 - val_accuracy: 0.8042 - val_loss: 0.4306
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 47s 75ms/step - accuracy: 0.8786 - loss: 0.3055 - val_accuracy: 0.8070 - val_loss: 0.4538
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 68ms/step - accuracy: 0.9294 - loss: 0.1964 - val_accuracy: 0.8216 - val_loss: 0.4669
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.9495 - loss: 0.1532 - val_accuracy: 0.8042 - val_loss: 0.5236


In [74]:
def conv(sample):
    d = imdb.get_word_index()
    sample = sample.lower().split()
    sample = [d.get(i) for i in sample]
    sample = pad_sequences([sample],padding='pre',maxlen=sent_len)
    return sample

def pred_sentiment(review):
    pred = model.predict(conv(review))
    if pred[0][0]>0.65:
        return 'Positive',pred[0][0]
    else:
        return 'Negative',pred[0][0]

print(pred_sentiment("Nolan is bad director and movie is bad"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
('Negative', 0.26441705)


In [75]:
model.save("rnn.h5")